It's not too hard to make a simple audio synthesizer in a Jupyter notebook.
IPywidgets has an [audio widget] for playing back sounds in the browser.

[audio widget]: https://ipywidgets.readthedocs.io/en/stable/examples/Beat%20Frequencies.html


In [ ]:
from functools import lru_cache # This import is just for speed
import ipywidgets
import IPython.display
from fractions import Fraction
import numpy as np
import os

In [ ]:
def make_audio_widget(data, sample_rate):
    return IPython.display.Audio(data, rate=sample_rate)

In [ ]:
def sin_series(basefrequency, timearray, coefficients, max_coefficients=20):
    """
    Inputs:
    -------
    basefrequency: Number. The base frequency (in Hertz) of the sound.
    timearray: Numpy array of shape(n,). The time at each sample.
    coefficients: Either a list of coefficients or a 
        function that maps coefficient indeces to coefficient values.
    """
    x = np.zeros_like(timearray)
    try:
        for coefficient in coefficients:
            x += coefficient*np.sin(
                2*np.pi*timearray*basefrequency*coefficient
            )
    except TypeError:
        for coefficient in range(1, max_coefficients):
            # print(coefficients(coefficient))
            x += coefficients(coefficient)*np.sin(
                2*np.pi*timearray*basefrequency*coefficient
            )
    x /= np.max(x) # Normalize the waveform
    return x

In [ ]:
def cancel_clicks(x, rampcount):
    anticlick = np.ones_like(x)
    ramp = np.linspace(0, 1, rampcount)
    anticlick[:rampcount] = ramp
    x *= anticlick
    return None

In [ ]:
@lru_cache()
def string_tone(
    pitch, 
    tone=1, 
    volume=1, 
    fade=1, 
    sr=44010, 
    final_loudness=0.001,
):
    duration = -fade * np.log(0.001)
    t = np.linspace(0, duration, int(duration*sr), endpoint = False)
    rampcount = int(sr*0.005)
    x = sin_series(
        basefrequency=pitch,
        timearray=t,
        coefficients=lambda x: 1/(np.exp(tone*x))
    )
    x = 0.25*x / np.max(x)
    x *= np.exp(-t/fade)
    cancel_clicks(x, rampcount)
    x *= volume
    return x

In [ ]:
def make_track(Notes, SampleRate,):
    Times = [time for sound, time in Notes]
    Track = np.zeros(
        int(max(Times) * SampleRate)
        + Notes[-1][0].shape[0]
    )
    for Note, Time in Notes:
        StartIndex = int(SampleRate * Time)
        EndIndex = StartIndex + Note.shape[0]
        Track[StartIndex:EndIndex] += Note
    return Track

In [ ]:
def get_pitch(degree, octave, basefreq=32.70):
    degree = int(degree)
    octave = int(octave) - 1
    return 2**(degree/12 + octave) * basefreq

In [ ]:
def get_pitch_from_string(s):
    octave = int(s[0], base=16)
    degree = int(s[1], base=16)
    return get_pitch(degree, octave)

In [ ]:
def get_duration_from_string(s):
    if len(s) > 2:
        duration = float(s[2:])
    else:
        duration = 1
    return duration

In [ ]:
def play_simple_song(
    Notes,
    BaseDuration = 0.25,
    NoteFade = 1,
    SampleRate = 44010,
    basefreq=32.70,
    **kwargs
):
    Progress = ipywidgets.Text(description='progress: ')
    length = len(Notes)
    IPython.display.display(Progress)
    NoteArrays = []
    Time = 0
    for i, NoteString in enumerate(Notes):
        Pitch = get_pitch_from_string(NoteString)
        Duration = get_duration_from_string(NoteString) * BaseDuration
        NoteArray = string_tone(
            Pitch, 
            fade=NoteFade,
            sr=SampleRate,
            
            **kwargs
        )
        NoteArrays.append((NoteArray, Time))
        Time += Duration
        Progress.value = f'{i} / {length}'
    Track = make_track(NoteArrays, SampleRate)
    return make_audio_widget(Track, SampleRate)

In [ ]:
Prelude = (
     '20 27 34 32 34 27 34 27 '*2
    +'20 29 35 34 35 29 35 29 '*2
    +'20 2B 35 34 35 2B 35 2B '*2
    +'20 30 34 32 34 30 34 30 '
    +'20 30 34 32 34 30 34 2B '
    +'20 29 34 32 34 30 2B 30 '
    +'29 30 2b 30 24 27 26 24 '
    +'26 30 32 30 32 30 32 30 '*2
    +'2b 32 37 36 37 32 30 32 '
    +'2b 32 30 32 27 2b 29 27 '
    +'19 24 30 2B 30 24 30 24 '*2
    +'19 26 27 29 27 26 24 22 '
    +'30 2b 29 37 36 34 32 30 '
    +'2b 29 27 37 32 37 2b 32 '
    +'27 29 2b 32 30 2b 29 27 '
    +'31 27 2a 29 2a 27 31 27 '
    +'34 27 2a 29 2a 27 31 27 '
    +'25 29 32 34 35 32 29 27 '
    +'25 29 32 34 35 32 2b 29 '
    +'28 2b 28 2b 32 2b 32 2b '*2
    +'30 2b 29 30 2b 30 32 2b '
    +'30 2b 29 27 25 24 22 20 '
    +'1b 25 27 25 27 25 27 25 '*2
    +'20 24 2a 29 2a 24 2a 24 '*2
    +'20 25 29 27 29 25 29 25 '*2
    +'20 2b 35 34 35 2b 35 2b '*2
    +'20 30 34 32 34 30 2b 29 '
    +'27 25 24 22 20 1b 19 17 '
    +'16 22 29 2b 30 29 2b 30 '*2
    +'15 22 27 29 2b 27 29 2b '*2
    +'15 22 27 2b 32 36 378   '
    +'22 24 25 27 29 2b 30 32 '
    +'2b 27 29 2b 30 32 34 35 '
    +'32 2b 30 32 34 35 37 382 '
    +'   37 36 37 37 35 34 35 '
    +'35 32 2b 29 27 22 24 25 '
    +'17 1b 22 27 32 34 35 32 '
    +'34 30 27 25 24 20 22 24 '
    +'17 20 24 27 30 32 34 30 '
    +'36 33 32 33 33 32 31 32 '
    +'32 30 2b 30 30 29 26 24 '
    +'22 26 29 30 32 36 37 36 '
    +'37 32 2b 29 2b 32 27 2b '
    +'22 27 26 24 22 20 1b 19 '
    +'178                     '
    +'373      36 34 32 30 2b '
    +'29 37 36 34 32 30 2b 29 '
    +'27 36 34 32 30 2b 29 27 '
    +'25 34 32 30 2b 29 27 25 '
    +'24 32 30 2b 29 27 25 24 '
    +'22 30 2b 29 2b 32 27 32 '
    +'29 32 2b 32 30 32 29 32 '
    +'2b 32 27 32 30 32 29 32 '*2
    +'2b 32 27 32 29 32 2b 32 '
    +'30 32 32 32 34 32 27 32 '
    +'32 32 34 32 35 32 27 32 '
    +'34 32 35 32 37 32 34 32 '
    +'35 32 34 32 35 32 32 32 '
    +'34 32 32 32 34 32 30 32 '
    +'32 32 30 32 32 32 2b 32 '
    +'30 32 2b 32 30 32 29 32 '
    +'2b 32 27 29 2a 27 2b 27 '
    +'30 27 31 27 32 27 33 27 '
    +'34 27 35 27 36 27 37 27 '
    +'38 27 39 27 3a 27 3b 27 '
    +'40 34 27 34 40 34 40 34 '
    +'40 34 27 34 40 34 40 34 '
    +'40 32 27 32 40 32 40 32 '
    +'40 32 27 32 40 32 40 32 '
    +'3b 35 27 35 3b 35 3b 35 '
    +'3b 35 27 35 3b 35 3b 35 '
    +'408'

).split()

In [ ]:
play_simple_song(
    Prelude, 
    BaseDuration=.2, 
    NoteFade=.3, 
    volume=0.2, 
    tone=0.3,
    
)